Import necessary libraries

In [2]:
from langchain_ollama import OllamaLLM,OllamaEmbeddings

Initialize the LLM

In [3]:
LLM_MODEL = 'llama3.2:1b'

llm = OllamaLLM(model=LLM_MODEL)

In [5]:
import os
import json
from langchain_core.documents import Document

DATA_DIR = os.path.join(os.getcwd(), "Data")

def load_json_docs():
    docs = []

    for file in os.listdir(DATA_DIR):
        if file.endswith(".json"):                 # Only process JSON files
            file_path = os.path.join(DATA_DIR, file)

            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)

                products = data.get("products", [])

                for product in products:
                    text = json.dumps(product, indent=2)

                    docs.append(
                        Document(
                            page_content=text,
                            metadata={
                                "id": product["id"],
                                "name": product["name"],
                                "category": product["category"],
                            }
                        )
                    )
    return docs

raw_docs = load_json_docs()
print(len(raw_docs))
print(raw_docs)

9
[Document(metadata={'id': 'prod-0001', 'name': 'AsterPhone Pro 14 Plus', 'category': 'smartphone'}, page_content='{\n  "id": "prod-0001",\n  "sku": "PH-APL-14P-256-BK",\n  "name": "AsterPhone Pro 14 Plus",\n  "category": "smartphone",\n  "brand": "Aster",\n  "model": "Pro 14 Plus",\n  "price": 1199.0,\n  "currency": "USD",\n  "release_date": "2024-09-15",\n  "warranty_months": 24,\n  "rating": 4.6,\n  "reviews_count": 1245,\n  "tags": [\n    "flagship",\n    "5g",\n    "oled",\n    "fast-charge"\n  ],\n  "images": [\n    "https://example.com/images/prod-0001-1.jpg"\n  ],\n  "variants": [\n    {\n      "color": "Midnight Black",\n      "storage_gb": 256\n    },\n    {\n      "color": "Starlight Silver",\n      "storage_gb": 512\n    }\n  ],\n  "specs": {\n    "os": "AOS 14",\n    "display": {\n      "size_in": 6.7,\n      "type": "LTPO OLED",\n      "resolution": "3200x1440",\n      "refresh_hz": 120\n    },\n    "processor": {\n      "name": "Aster A18",\n      "cores": 8,\n      "fr

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=50
)

documents = splitter.split_documents(raw_docs)
print(len(documents))
print(documents)

43
[Document(metadata={'id': 'prod-0001', 'name': 'AsterPhone Pro 14 Plus', 'category': 'smartphone'}, page_content='{\n  "id": "prod-0001",\n  "sku": "PH-APL-14P-256-BK",\n  "name": "AsterPhone Pro 14 Plus",\n  "category": "smartphone",\n  "brand": "Aster",\n  "model": "Pro 14 Plus",\n  "price": 1199.0,\n  "currency": "USD",\n  "release_date": "2024-09-15",\n  "warranty_months": 24,\n  "rating": 4.6,\n  "reviews_count": 1245,\n  "tags": [\n    "flagship",\n    "5g",\n    "oled",\n    "fast-charge"\n  ],\n  "images": ['), Document(metadata={'id': 'prod-0001', 'name': 'AsterPhone Pro 14 Plus', 'category': 'smartphone'}, page_content='"oled",\n    "fast-charge"\n  ],\n  "images": [\n    "https://example.com/images/prod-0001-1.jpg"\n  ],\n  "variants": [\n    {\n      "color": "Midnight Black",\n      "storage_gb": 256\n    },\n    {\n      "color": "Starlight Silver",\n      "storage_gb": 512\n    }\n  ],\n  "specs": {\n    "os": "AOS 14",\n    "display": {\n      "size_in": 6.7,\n      

In [7]:
from langchain_community.vectorstores import Chroma

EMBEDDING_MODEL = 'nomic-embed-text'

CHORMA_DB_DIR = os.path.join(os.getcwd(), "Vectorstore")

embeddings = OllamaEmbeddings(model=EMBEDDING_MODEL)
vectordb = Chroma.from_documents(
    documents,
    embedding=embeddings,
    persist_directory=CHORMA_DB_DIR
)

vectordb.persist()

print("Vector DB Created.")

Vector DB Created.


C:\Users\Dulitha Harasara\AppData\Local\Temp\ipykernel_2704\3698220226.py:14: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Define a message template for the chatbot
message = """
You are a intelligent chatbot of the online store.Answer this question using the provided context only.
you need follow following rules.
1. if only someone ask from you recommod a product you need to recommod products with its full details from the store.
2. If the information is NOT present in the context, say: "Sorry, I don't have information about that product."  
3. Do NOT make up or hallucinate product details.  
4. Always respond in a helpful and polite tone.  
5. If the user asks for something unrelated (for example: jokes, coding, math), reply:  
   "I am an online store assistant. I can help you with product information, availability, and recommendations."  
6. If the user asks for comparisons, provide a comparison using ONLY the context.  
7. If the user asks for price, availability, or specifications, give details ONLY from the context.  
8. If the user asks for alternatives, recommend similar products from the context.  
9. If the user asks a question with missing details, ask a clarifying question first.  
10. Never answer using your own external knowledge — use ONLY the context below.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human",message)])
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\nYou are a intelligent chatbot of the online store.Answer this question using the provided context only.\nyou need follow following rules.\n1. if only someone ask from you recommod a product you need to recommod products with its full details from the store.\n2. If the information is NOT present in the context, say: "Sorry, I don\'t have information about that product."  \n3. Do NOT make up or hallucinate product details.  \n4. Always respond in a helpful and polite tone.  \n5. If the user asks for something unrelated (for example: jokes, coding, math), reply:  \n   "I am an online store assistant. I can help you with product information, availability, and recommendations."  \n6. If the user asks for comparisons, provide a comparison using ONL

In [9]:
CHORMA_DB_DIR = os.path.join(os.getcwd(), "Vectorstore")

db = Chroma(
    persist_directory=CHORMA_DB_DIR,
    embedding_function=embeddings
)

retriever = db.as_retriever(search_kwargs={"k": 4})

C:\Users\Dulitha Harasara\AppData\Local\Temp\ipykernel_2704\1621540715.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  db = Chroma(


In [10]:
chain = ({"context":retriever,"question":RunnablePassthrough()} | prompt | llm)

In [11]:
response = chain.invoke("what is a car?")
print(response) 

I am an online store assistant. I can help you with product information, availability, and recommendations.

What is a car?

The provided context does not mention cars. Could you please provide more details or clarify what you are looking for?
